Instalacja Pulpa i Sudo

In [1]:
!pip install pulp

!sudo apt-get install coinor-cbc glpk-utils coinor-clp

     |████████████████████████████████| 40.6MB 112kB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  coinor-libcbc3 coinor-libcgl1 coinor-libclp1 coinor-libcoinutils3v5
  coinor-libosi1v5 libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev default-libmysqlclient-dev
The following NEW packages will be installed:
  coinor-cbc coinor-clp coinor-libcbc3 coinor-libcgl1 coinor-libclp1
  coinor-libcoinutils3v5 coinor-libosi1v5 glpk-utils libamd2 libcolamd2
  libglpk40 libsuitesparseconfig5
0 upgraded, 12 newly installed, 0 to remove and 14 not upgraded.
Need to get 3,439 kB of archives.
After this operation, 9,832 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libsuitesparseconfig5 amd64 1:5.1.2-2 [9,044 B]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libamd2 amd64 1:5.1.2-2 [19.5 kB]
Get:3 

Import Pulpa oraz sprawdzenie wersji.

In [3]:
import pulp
print(pulp.__version__)

2.3.1


**Zadanie 1 - Wyznaczenie maksymalnego prądu płynącego przez układ.**

Importowanie Pulpa.

In [4]:
from pulp import *

Zdefiniowanie problemu poprzez zmienną "prob".

In [5]:
prob = LpProblem("MaxPrad",LpMaximize)

Zdefiniowanie 5 zmiennych odpowiadających prądom płynących przez układ.

In [6]:
I1 = LpVariable("I1",0,None,LpContinuous)
I2 = LpVariable("I2",0,None,LpContinuous)
I3 = LpVariable("I3",0,None,LpContinuous)
I4 = LpVariable("I4",0,None,LpContinuous)
I5 = LpVariable("I5",0,None,LpContinuous)

Stworzenie równań zgodnych z treścią zadania.

In [7]:
prob += I3, "SumaPradow" 

prob += I1 + I2 == I3, "suma1"
prob += I4 + I5 == I3, "suma2"


prob += I1*8 == I2*6, "U1"
prob += I5*8 == I4*10, "U2"

Wprowadzenie ograniczeń prądów.

In [8]:
prob += I1 <= 2, "ogr_I1"
prob += I2 <= 3, "ogr_I2"
prob += I3 <= 4, "ogr_I3"
prob += I4 <= 2, "ogr_I4"
prob += I5 <= 2, "ogr_I5"

Zapisanie problemu do pliku .lp oraz rozwiązanie problemu za pomocą Solvera wybranego przez Pulpa.

In [9]:
prob.writeLP("MaxPrad.lp")

prob.solve()

1

Wyprintowanie statusu rozwiązania oraz zmiennych z rozwiązaną optymalną wartością.

In [10]:
print ("Status:", LpStatus[prob.status])

print("Max prad układu:",prob.objective.value(),"A")
print("Wartości poszczególnych prądów wynoszą:")
for v in prob.variables():
    print (v.name, "=", v.varValue,"A")

Status: Optimal
Max prad układu: 3.6 A
Wartości poszczególnych prądów wynoszą:
I1 = 1.5428571 A
I2 = 2.0571429 A
I3 = 3.6 A
I4 = 1.6 A
I5 = 2.0 A


Program działa poprawnie, status rozwiązania jest optymalny.

**Zadanie 2 - Wyznaczenie minimalnej mocy rozproszenia na rezystorach oraz wartości prądów.**

Importowanie Pulpa.

In [11]:
from pulp import *

Przypisanie napięciom ich wartości zgonych z treścią zadania. Napięcia będą potrzebne do wyznaczenia wartości poszczególnych rezystancji na mostku.

In [12]:
U1=6
U2=10
U3=4
U4=7
U5 =3

Zdefiniowanie problemu w zmiennej "prob".

In [13]:
prob = LpProblem("MinMoc",LpMinimize)

Zdefiniowanie zmiennych.

In [14]:
I1=LpVariable("I1",0)
I2=LpVariable("I2",0)
I3=LpVariable("I3",0)
I4=LpVariable("I4",0)
I5=LpVariable("I5",0)

Stworzenie funkcji dotyczącej wyznaczenia minimalnej mocy.

In [15]:
prob += 6*I1 + 10*I2 + 4*I3 + 7*I4 + 3*I5
prob += I1+I2 == I4+I5
prob += I1 == I3+I4
prob += I2 == I5-I3

Wprowadzenia ograniczeń prądów.

In [16]:
prob += I1 >= 3, "I1min"
prob += I1 <= 5, "I1max"
prob += I2 >= 1, "I2min"
prob += I2 <= 3, "I2max" 
prob += I3 >= 1, "I3min"
prob += I3 <= 3, "I3max"
prob += I4 >= 1, "I4min"
prob += I4 <= 3, "I4max"
prob += I5 >= 3, "I5min"
prob += I5 <= 5, "I5max"

Zapisanie problemu do pliku .lp oraz rozwiązanie numeryczne problemu.

In [17]:
prob.writeLP("MinMoc.lp")
prob.solve()

1

Wyprintowanie statusu rozwiązania oraz maksymalnej mocy na mostku.

In [18]:
print("Status:", LpStatus[prob.status])

print("Min moc: {} mW".format(value(prob.objective)))

Status: Optimal
Min moc: 52.0 mW


Wyprintowanie zmiennych z rozwiązaną optymalną wartością.

In [19]:
for v in prob.variables():
    print (v.name, "=", v.varValue,"mA")

I1 = 3.0 mA
I2 = 1.0 mA
I3 = 2.0 mA
I4 = 1.0 mA
I5 = 3.0 mA


Wyprintowanie wartości poszczególnych rezystancji na mostku, korzystając ze wzoru Ohma.

In [20]:
print("R1 =", U1/I1.varValue,"kOhm")
print("R2 =", U2/I2.varValue,"kOhm")
print("R3 =", U3/I3.varValue,"kOhm")
print("R4 =", U4/I4.varValue,"kOhm")
print("R5 =", U5/I5.varValue,"kOhm")

R1 = 2.0 kOhm
R2 = 10.0 kOhm
R3 = 2.0 kOhm
R4 = 7.0 kOhm
R5 = 1.0 kOhm


Program działa poprawnie, status rozwiązania jest optymalny.